To open on Google Colab [link](https://colab.research.google.com/github/RodrigoAVargasHdz/CHEM-4PB3/blob/main/Course_Notes/Week7/gradentdescent_torch.ipynb)


In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch import nn
import pandas as pd

# Linear model (recap)

In the previous lectures, we studied linear model,\
$f_{\phi(\cdot)}(\mathbf{x}) = \mathbf{w}^\top \phi(\mathbf{x}) = \sum_i w_i \phi_i(\mathbf{x})$.

* $\phi(\cdot)$ is the linear map to a new **feature representation**, for example, polynomials, Fourier basis, ec.

For a square loss function, $ {\cal L} \approx \|\mathbf{y} - f_{\phi(\cdot)}(\mathbf{X}) \|_2$, the optimal parameters $\mathbf{w}^*$ can be obtained in closed form.\

$\mathbf{w}^* = \Big( \Phi(\mathbf{X})^\top\Phi(\mathbf{X})\Big)^{-1} \Phi(\mathbf{X})^\top\mathbf{y}$.

Theoretically, the bottleneck to solve for $\mathbf{w}^*$ relies on inverting the matrix $\Big( \Phi(\mathbf{X})^\top\Phi(\mathbf{X})\Big)$.\
Matrix inversion is one of the most **expensive operations**, ${\cal O}(N^3)$ where $N$ is this case is the number of training points.

Let's assume we have a **GIGANTIC** dataset, $N \approx 100,000$ points. 
It will computationally demanding to solve for $\mathbf{w}^*$. 

However, the gradient of ${\cal L}(\mathbf{w})$ with respect to $\mathbf{w}$ scales **quadratically**,\
$\nabla {\cal L}(\mathbf{w}) = \Phi(\mathbf{X})^\top\Phi(\mathbf{X})\mathbf{w} -\Phi(\mathbf{X})^\top\mathbf{y}$\
Matrix-matrix multiplication $(\Phi(\mathbf{X})^\top\Phi(\mathbf{X}))$ scales  ${\cal O}(N^2)$, (**one order of magnitude lower**).


* **$N = 100, \;\; N^2 = 10,000, \;\; N^3 = 1,000,000$**


## Gradient descent

We can search for the minimum/maximum of $\nabla {\cal L}(\mathbf{w})$ using gradient-based methods.\
For example, for $f(x_0,x_1) = x_0^2 + x_1^2$, 

In [ ]:
f_sqr = lambda x:np.sum(np.power(x,2),axis=1)
x = np.linspace(-2., 2., 50)
x0,x1 = np.meshgrid(x,x)
X = np.column_stack((x0.flatten(),x1.flatten()))
y = f_sqr(X)
y = y.reshape((x0.shape))

In [ ]:
plt.title(f'$f(x_0,x_1) = x_0^2 + x_1^2$')
plt.contourf(x0, x1, y, levels=60)
plt.xlabel(f'$x_0$', fontsize=15)
plt.ylabel(f'$x_1$', fontsize=15)

# Gradients 
The gradient (Jacobian) of $f(x_0,x_1) = x_0^2 + x_1^2$ is,\
$\frac{\partial f}{\partial x_0} = 2x_0$ and $\frac{\partial f}{\partial x_1} = 2x_1$.\
Meaning, $\nabla f(x_0,x_1) = [2x_0,2x_1]$

At any given point, $\nabla f(x_0,x_1)$ tells you the direction in which the function changes with the greatest rate.\
If you think of the function as height, then it gives the direction in which the ground is steepest.

In [ ]:
grad_f_sqr = lambda x:2*x

gx = np.random.uniform(low=-0.75,high=0.75,size=(10,2))
gx0,gx1 = gx[:,0],gx[:,1]
gX = np.column_stack((gx0.flatten(), gx1.flatten()))
print(gX.shape)
grad = grad_f_sqr(gX)

In [ ]:
# gradient field
fig, ax = plt.subplots(1, 1,figsize=(8,8))
ax.set_aspect(1)
# ax.plot(feature_x, feature_y, c='k')
gf_x0 = grad[:,0].reshape(gx0.shape)
gf_x1 = grad[:,1].reshape(gx1.shape)
ax.contourf(x0, x1, y, lw=2, levels=60)

eta = 0.1 #rescale
ax.quiver(gx0, gx1, eta*gf_x0, eta*gf_x1,
          units='xy', scale=0.15, color='white')
ax.set_aspect('equal')
ax.set_title(r'$-\nabla f(x_0,x_1)$')
ax.set_xlabel(r'$\frac{\partial f}{\partial x_{0}}$', fontsize=15)
ax.set_ylabel(r'$\frac{\partial f}{\partial x_{1}}$', fontsize=15)


# Minimum search using gradient descent
Algorithm
1. Start with a random initial value of $x_0$ and $x_1$; $[x_0,x_1]_{t}$
2. Compute the gradient; $\nabla f(x_0,x_1) = \nabla f([x_0,x_1]_{t})$
3. Update $x_0$ and $x_1$ using the information from step 2.
    * $[x_0,x_1]_{t+1} = [x_0,x_1]_t - \eta \nabla f(x_0,x_1)$ 
  * $\eta$ is known as the step size or **learning rate**
1. Repeat until convergence criteria is is satisfied:
    * $\|\nabla f(x_0,x_1) \| < 10^{-6}$
    *  $\| f([x_0,x_1]_{t+1}) - f([x_0,x_1]_{t})\| < 10^{-6} $
 * $10^{-6}$ is a threshold  parameter, and is problem dependent.

In [ ]:
# gradient descent algorithm
x = np.random.uniform(-2.,2.,(1,2))
eta = 0.1

x_trajectory = np.zeros_like(x.ravel())
g_trajectory = np.zeros_like(x.ravel())
f_trajectory = np.zeros(1)
c = 1E-4
xt = x
for t in range(20):
    grad_t = grad_f_sqr(xt)
    y_t = f_sqr(xt)
    g_trajectory = np.vstack((g_trajectory,grad_t))
    f_trajectory = np.append(f_trajectory, y_t)
    x_trajectory = np.vstack((x_trajectory, xt))
    print('t = %s, f = %.4f, '%(t,y_t),'x = ', x   ,'g = ', grad_t)
    xt = xt - eta*grad_t
    
    if np.linalg.norm(g_trajectory[-2] - g_trajectory[-1]) < c or np.linalg.norm(f_trajectory[-2] - f_trajectory[-1]) < c:
        break

# remove initial useless point.
g_trajectory = g_trajectory[1:]
f_trajectory = f_trajectory[1:]
x_trajectory = x_trajectory[1:]

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(8,8))
ax.set_aspect(1)
# ax.plot(feature_x, feature_y, c='k')
gf_x0 = grad[:,0].reshape(gx0.shape)
gf_x1 = grad[:,1].reshape(gx1.shape)
ax.contourf(x0, x1, y, lw=2, levels=60)
ax.plot(x_trajectory[:,0],x_trajectory[:,1],color='red',marker='o',label=r'$[x_0,x_1]_t$')
ax.set_aspect('equal')
ax.set_xlabel(r'$x_0$',fontsize=15)
ax.set_ylabel(r'$x_1$',fontsize=15)
plt.legend()

## What happen when we do not have gradients $\nabla f(\mathbf{x})$?

(extra topic), study [Finite Difference](https://mathworld.wolfram.com/FiniteDifference.html)

Finite Difference --> numerical approximation for gradients and higher order derivatives.

## Let's revise **linear model**,
$f_{\phi(\cdot)}(\mathbf{x}) = \mathbf{w}^\top \phi(\mathbf{x}) = \sum_i w_i \phi_i(\mathbf{x})$.

**Diagram**\
<img src="https://raw.github.com/RodrigoAVargasHdz/CHEM-4PB3/master/Course_Notes/Figures/Linear_model_diagram.png"  width="350" height="300">


## Linear model in Torch for Harmonic series

Harmonic series to approximate the Morse function. [**Reference**](https://www.tandfonline.com/doi/full/10.1080/08927020903433739)\
**Morse function**\
$E_{Morse} = D_{e}\left (1-\exp^{-\alpha(r-r_0)} \right )^2$,\
where $\alpha = \sqrt{k/2D_{e}}$.

**Harmonic series**\
$E_{harmonic} = \sum_{\ell=1}^p \frac{k_\ell}{r^{\ell}}$, 
is alternative to the polynomials that approaches to zero as $r$ approaches to infinity.

Computationally, $E_{harmonic}$ is simply a linear model over $[r,r^1,\cdots,r^\ell]$, and the parameters $k_\ell$ are linear weigths.\
(let's code this in torch)

```python
def polynomial(r,ell):
    r = 1/r #power ell = 1
    R = r
    # loop over higher orders
    for i in range(2,ell+1):
        r_l = np.power(r,i)
        R = np.column_stack((R,r_l))
    return R
```

In [ ]:
# code in torch
class HarmonicSeries(nn.Module):
    def __init__(self, ell):
        super().__init__()
        self.ell = ell

    def forward(self, x):
        x = torch.div(1,x)
        R = x
        for i in range(2,self.ell+1):
            r_l = x.pow(i)
            R = torch.column_stack((R,r_l))
        return R

In [ ]:
class LinearHarmonicModel(nn.Module):
    def __init__(self,ell):
        super().__init__()
        self.ell = ell
        self.harmonic = HarmonicSeries(self.ell)
        self.linear = nn.Linear(self.ell,1)
        nn.init.zeros_(self.linear.weight) #play around with this initialization

    def forward(self, x):
        r = self.harmonic(x)
        y = self.linear(r)
        return y

## Fit N2 PES (again)

In [ ]:
#data
data_url = "https://github.com/RodrigoAVargasHdz/CHEM-4PB3/raw/main/Course_Notes/data/PES_N2"
data = pd.read_csv(data_url)
X, y = data['R'].to_numpy(), data['Energy'].to_numpy()
X = torch.from_numpy(X).unsqueeze(1).float()
y = (y - np.mean(y))/np.std(y)
y = torch.from_numpy(y).unsqueeze(1).float()
print(X.shape,y.shape)

In [ ]:
# test our model
model = LinearHarmonicModel(20)
X_rnd = torch.randn((10, 1))
y_pred = model(X)
loss = nn.MSELoss()
print(loss(y_pred,y))
print(y_pred.shape)
print('Parameters')
print(model.linear.weight)
print(model.linear.bias)


plt.plot(X.numpy(),y.numpy())
plt.plot(X.numpy(),model(X).detach().numpy())
plt.xlabel('R(N-N)')
plt.ylabel('Energy')

In [ ]:
#define a linear model
m = nn.Linear(10,2) # input dimensions -> output dimensions
print(m.weight)
print(m.bias)
X = torch.randn((5,10))
y = X @ m.weight.T
print(y)

In [ ]:
# Training

# Use the adam optimizer
# Includes GaussianLikelihood parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# "Loss" for GPs - the marginal log likelihood
loss = nn.MSELoss()

training_iter = 2000
loss_trajectory = []
loss_trajectory_tst = []

l2_lambda = 0.1
for itr in range(1,training_iter):
    output = model(X)
    loss_val = loss(output,y)
    
    # l_norm = sum(p.pow(2.0).sum()
    #               for p in model.parameters())
    l_norm = sum(p.abs().sum()for p in model.parameters())
    
    loss_val = loss_val + l2_lambda*l_norm
    
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()
    
    if itr%10 == 0 :
        print(f'itr = %s, loss = %.4f'%(itr,loss_val.item()))

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(X.numpy()[::2], y.numpy()[::2], marker='s', markersize=5,label='DFT')

X_grid = torch.linspace(X.min(),X.max(),2000)
X_grid = X_grid.unsqueeze(-1)
plt.plot(X_grid.detach().numpy(), model(X_grid).detach().numpy(),label='Harmonic Series')
plt.xlabel('R(N-N)',fontsize=15)
plt.ylabel('Energy',fontsize=15)
plt.legend()


## Next lecture

## What if $\phi(\cdot)$ also depends on internal paramters? **non-linear model**,
$f(\mathbf{x}) = \mathbf{w}^\top \phi(\mathbf{x},\mathbf{w}') = \sum_i w_i \phi_i(\mathbf{x},\mathbf{w}')$.

Now we also need to optimize the non-linear parameters $\mathbf{w}'$.

**Diagram**\
<img src="https://raw.github.com/RodrigoAVargasHdz/CHEM-4PB3/master/Course_Notes/Figures/nonLinear_model_diagram.png"  width="350" height="300">
